# Scenario Analysis and Stress Testing
In this notebook for exercises to do with VaR I make use of a separate Python module `hist_simulation`, which you can find under the `ch13` directory of this project. Essentially all logic to sole the exercises is implemented in it.

In [1]:
from ch13 import hist_simulation

import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import Bounds

from math import exp
from functools import partial

In [2]:
# All amounts passed are in thousands of dollars
hs = hist_simulation.HistSimulation('./ext/VaRExampleRMFI3eHistoricalSimulation.xls', 1e4)

In [3]:
import locale
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

#### Exercise 22.8

In [4]:
hs.exercise_22_8([(240, .005), (280, .005), (340, .002), (500, .002),
                  (700, 5e-4), (850, 5e-4), (1050, 5e-4)])

Exercise 22.8. One day VaR with a confidence level of 99.0%: $282,203.85
Exercise 22.8. One day ES with a confidence level of 99.0%: $609,039.44


#### Exercise 22.9
This exercise demonstrates _reverse stress testing_. Answer from the textbook:
* Worst-case scenario is achieved when asset price is $60 and volatility σ is 30%

In [5]:
# Define d1, d2 as per the Black-Scholes-Merton formula for a futures or forward price
# Any of the arguments can be either a scalar or a numpy array
def  d1(S0, sigma, K, T, r=0.):
    return  (np.log(S0/K) + (r + np.square(sigma) / 2) * T) / (sigma * np.sqrt(T))
def  d2(S0, sigma, K, T, r=0.):
    return  d1(S0, sigma, K, T, r) - sigma * np.sqrt(T)

In [6]:
# Define the Black-Scholes-Merton formula
def call_opt(S0, sigma, K, T, r):
    return S0*norm.cdf(d1(S0, sigma, K, T, r)) - K*exp(-risk_free_rate*T)*norm.cdf(d2(S0, sigma, K, T, r))

In [7]:
S0 = 50.
risk_free_rate = .03 # continuous compounding
sigma = .2
positions = np.array([200, -70, -120, -60]) # * 1000
strikes = np.array([50, 60, 40, 55])
Ts = np.array([1., 1.5, .8, .5])
df = pd.DataFrame(np.array([positions, strikes, Ts]).T, columns=['Position', 'Strike Price', 'Life (years)'])

First let's calculate positions' values

In [8]:
def price_position(row, S0, sigma):
    '''
    :param row: a pd.Series object representing an option position, expected values are position, K, T
    '''
    return row[0] * call_opt(S0, sigma, row[1], row[2], risk_free_rate)

def price_positions(df, S0, sigma):
    return df.apply(partial(price_position, S0=S0, sigma=sigma), axis=1).sum()

In [9]:
df['Position Value'] = df.apply(partial(price_position, S0=S0, sigma=sigma), axis=1)
df

,Position,Strike Price,Life (years),Position Value
0,200.0,50.0,1.0,941.340338
1,-70.0,60.0,1.5,-164.391186
2,-120.0,40.0,0.8,-1349.943909
3,-60.0,55.0,0.5,-78.357066


Now we are ready to define an objective function

In [10]:
cur_portfolio_value = df.iloc[:,-1].sum()
print('Portfolio value: {:s}'.format(locale.currency(cur_portfolio_value, grouping=True)))

Portfolio value: -$651.35


In [11]:
def objective_function(x):
    price, volatility = x
    return price_positions(df, price, volatility)

In [12]:
x0 = (S0, sigma)        # starting with the current asset price and volatility
# S0[40, 60), sigma[.1, .3]
bounds = Bounds([40., .1], [60., .3])
res = minimize(objective_function, x0, bounds=bounds, method = 'Nelder-Mead')
res = minimize(objective_function, x0, bounds=bounds, method = 'Powell')
if res.success:
    print('Worst case scenario:\n\tS0={:s}, \u03C3={:.2%}'
          .format(locale.currency(res.x[0], grouping=True), res.x[1]))
    df.iloc[:,-1].update(df.apply(partial(price_position, S0=res.x[0], sigma=res.x[1]), axis=1))
    print('\tPortfolio Value: {:s}'.format(locale.currency(df.iloc[:,-1].sum(), grouping=True)))
    print('\tImplying a loss of: {:s}'.format(
        locale.currency(df.iloc[:,-1].sum()-cur_portfolio_value, grouping=True)))

Worst case scenario:
	S0=$60.00, σ=29.99%
	Portfolio Value: -$992.74
	Implying a loss of: -$341.39


#### Exercise 22.11
I assume the autor is referring to Section 13.3.3 "Volatility Scaling for the Portfolio" and solve it with an EWMA &lambda;=0.94.

In [13]:
hs.exercise_22_11(.94, [(235, .005), (300, .002), (450, .002), (750, 5e-4), (850, 5e-4)])

Exercise 22.11. One day VaR with a confidence level of 99.0% and λ=0.94: $627,915.82
Exercise 22.11. One day ES with a confidence level of 99.0% and λ=0.94: $785,029.94
